In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  

session = SparkSession.builder.getOrCreate()

schema = StructType([StructField('medallion',StringType(),True),
                     StructField('hack_license',StringType(),True),
                     StructField('vendor_id',StringType(),True),
                     StructField('rate_code',IntegerType(),True),
                     StructField('store_and_fwd_flag',StringType(),True),
                     StructField('passenger_count',IntegerType(),True),
                     StructField('trip_time_in_secs',IntegerType(),True),
                     StructField('trip_distance',DoubleType(),True),
                     StructField('pickup_longitude',DoubleType(),True),
                     StructField('pickup_latitude',DoubleType(),True),
                     StructField('dropoff_longitude',DoubleType(),True),
                     StructField('dropoff_latitude',DoubleType(),True)])

stream_df = (
  session
    .readStream    
    .schema(schema)             
    .option("maxFilesPerTrigger", 1) \
    .csv("trip_data_small_dir/*.csv")
)

# add processing time as a new column
time_stream_df = stream_df.withColumn("processingTime",current_timestamp()) 
import pyspark.sql.functions as F
import random
time_stream_df = time_stream_df.withColumn("processingTime", (F.unix_timestamp("processingTime") + F.rand() * 15).cast('timestamp'))

21/09/21 08:12:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/21 08:12:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/09/21 08:12:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/09/21 08:12:39 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
stream_count_df = (                 
  time_stream_df
    .groupBy(
      time_stream_df.passenger_count, 
      window(time_stream_df.processingTime, "5 seconds"))
    .count()
)

stream_df.isStreaming

True

In [3]:
stream_count_df.writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate","false") \
    .start() \
    .awaitTermination()

21/09/21 08:13:32 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-48ff3179-9d01-41c4-b391-a5123e4f46ae. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/09/21 08:13:35 WARN FileStreamSource: Listed 200 file(s) in 2054 ms          


-------------------------------------------
Batch: 0
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|1              |{2021-09-21 08:13:45, 2021-09-21 08:13:50}|576  |
|3              |{2021-09-21 08:13:35, 2021-09-21 08:13:40}|52   |
|6              |{2021-09-21 08:13:40, 2021-09-21 08:13:45}|50   |
|4              |{2021-09-21 08:13:45, 2021-09-21 08:13:50}|21   |
|5              |{2021-09-21 08:13:35, 2021-09-21 08:13:40}|140  |
|1              |{2021-09-21 08:13:35, 2021-09-21 08:13:40}|561  |
|5              |{2021-09-21 08:13:45, 2021-09-21 08:13:50}|133  |
|6              |{2021-09-21 08:13:35, 2021-09-21 08:13:40}|76   |
|4              |{2021-09-21 08:13:35, 2021-09-21 08:13:40}|23   |
|3              |{2021-09-21 08:13:45, 2021-09-21 08:13:50}|58   |
|5              |{2021-09-21 08:

-------------------------------------------
Batch: 1
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|5              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|112  |
|2              |{2021-09-21 08:13:50, 2021-09-21 08:13:55}|138  |
|2              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|101  |
|1              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|548  |
|4              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|19   |
|6              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|87   |
|6              |{2021-09-21 08:13:50, 2021-09-21 08:13:55}|78   |
|2              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|118  |
|6              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|70   |
|3              |{2021-09-21 08:13:50, 2021-09-21 08:13:55}|32   |
|3              |{2021-09-21 08:

-------------------------------------------
Batch: 2
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|5              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|154  |
|2              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|161  |
|4              |{2021-09-21 08:14:10, 2021-09-21 08:14:15}|7    |
|1              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|636  |
|1              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|790  |
|4              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|34   |
|3              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|38   |
|6              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|147  |
|2              |{2021-09-21 08:14:00, 2021-09-21 08:14:05}|252  |
|6              |{2021-09-21 08:13:55, 2021-09-21 08:14:00}|98   |
|4              |{2021-09-21 08:

-------------------------------------------
Batch: 3
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|4              |{2021-09-21 08:14:10, 2021-09-21 08:14:15}|27   |
|4              |{2021-09-21 08:14:15, 2021-09-21 08:14:20}|21   |
|1              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|1250 |
|3              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|76   |
|4              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|35   |
|5              |{2021-09-21 08:14:05, 2021-09-21 08:14:10}|215  |
|3              |{2021-09-21 08:14:15, 2021-09-21 08:14:20}|35   |
|5              |{2021-09-21 08:14:15, 2021-09-21 08:14:20}|124  |
|6              |{2021-09-21 08:14:10, 2021-09-21 08:14:15}|101  |
|2              |{2021-09-21 08:14:10, 2021-09-21 08:14:15}|229  |
|5              |{2021-09-21 08:

...

-------------------------------------------
Batch: 196
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|6              |{2021-09-21 08:26:00, 2021-09-21 08:26:05}|179  |
|1              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|728  |
|6              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|150  |
|3              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|100  |
|3              |{2021-09-21 08:26:00, 2021-09-21 08:26:05}|116  |
|5              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|234  |
|3              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|57   |
|1              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|1242 |
|2              |{2021-09-21 08:26:00, 2021-09-21 08:26:05}|347  |
|5              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|147  |
|4              |{2021-09-21 0

-------------------------------------------
Batch: 197
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|6              |{2021-09-21 08:26:00, 2021-09-21 08:26:05}|193  |
|1              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|1331 |
|6              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|198  |
|3              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|44   |
|3              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|139  |
|4              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|14   |
|3              |{2021-09-21 08:26:00, 2021-09-21 08:26:05}|120  |
|5              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|373  |
|3              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|105  |
|1              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|1829 |
|2              |{2021-09-21 0

-------------------------------------------
Batch: 198
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|1              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|1969 |
|6              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|236  |
|3              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|95   |
|3              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|169  |
|4              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|44   |
|5              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|452  |
|3              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|148  |
|1              |{2021-09-21 08:26:05, 2021-09-21 08:26:10}|2212 |
|4              |{2021-09-21 08:26:20, 2021-09-21 08:26:25}|10   |
|6              |{2021-09-21 08:26:20, 2021-09-21 08:26:25}|30   |
|5              |{2021-09-21 0

-------------------------------------------
Batch: 199
-------------------------------------------
+---------------+------------------------------------------+-----+
|passenger_count|window                                    |count|
+---------------+------------------------------------------+-----+
|1              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|2515 |
|3              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|142  |
|6              |{2021-09-21 08:26:25, 2021-09-21 08:26:30}|14   |
|4              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|66   |
|3              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|191  |
|4              |{2021-09-21 08:26:20, 2021-09-21 08:26:25}|32   |
|6              |{2021-09-21 08:26:20, 2021-09-21 08:26:25}|74   |
|2              |{2021-09-21 08:26:25, 2021-09-21 08:26:30}|27   |
|5              |{2021-09-21 08:26:10, 2021-09-21 08:26:15}|525  |
|2              |{2021-09-21 08:26:15, 2021-09-21 08:26:20}|407  |
|2              |{2021-09-21 0

21/09/21 08:43:44 WARN FileStreamSource: Listed 200 file(s) in 2022 ms          
